In [ ]:
'''
Author: Ngawang Gurung
Date: 2024/07/24
'''

In [1]:
import pandas as pd
from datetime import datetime
import time
import random
import os
from dotenv import load_dotenv

# This project-specific libaries
from kafka import KafkaProducer

# Own Libraries
from helper import aes_encrypt, table_df

In [2]:
df = table_df('client_rw', 'fc_account_master')
df.head()

,account_number,customer_code,product,product_category,acc_open_date,acc_closed_date,active_flag
0,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,None,0
1,02XYZXYZ10015593701,KL0255937,SBA,SBPPS,1999-03-10,None,0
2,02XYZXYZ10015593801,KL0255938,SBA,SBPPS,1999-03-10,None,0
3,02XYZXYZ10015594801,KL0255948,SBA,SBANU,1999-03-22,None,0
4,02XYZXYZ10015597601,KL0255976,SBA,SBPPS,1999-04-22,None,0


In [3]:
load_dotenv()

SUPER_SECRET_KEY = os.getenv('SUPER_SECRET_KEY') # Super secret key (must be 16, 24, or 32 bytes long)
df['account_number'] = df['account_number'].apply(lambda x: aes_encrypt(x, SUPER_SECRET_KEY))
df.head()

,account_number,customer_code,product,product_category,acc_open_date,acc_closed_date,active_flag
0,+Y7DNNNCRLxa9tHiluVGkYY5B2W/n3ZpLplkJINLIdM=,KL0255921,SBA,SBPPS,1999-03-10,None,0
1,U7RRmweke5W8oVRNvaKXpZJPf6MTWIf+3pVCi/uDj3w=,KL0255937,SBA,SBPPS,1999-03-10,None,0
2,U7RRmweke5W8oVRNvaKXpW7opsj+z4fEE98TMsDU5ZQ=,KL0255938,SBA,SBPPS,1999-03-10,None,0
3,Khed5di6hfdejAOOBP6XBG7opsj+z4fEE98TMsDU5ZQ=,KL0255948,SBA,SBANU,1999-03-22,None,0
4,wt0lfnWsGTalCHEtZvy671U78SwdSDGmCdZQ2GwRpAw=,KL0255976,SBA,SBPPS,1999-04-22,None,0


In [4]:
KAFKA_TOPIC = "am"
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

producer = KafkaProducer(bootstrap_servers = KAFKA_BOOTSTRAP_SERVERS, value_serializer = lambda x: x.encode('utf-8'))

In [5]:
df_list = df.to_dict(orient="records")

print("Kafka Producer Application Started ... ")

for message in df_list:
    message_data = ','.join(str(message[column]) for column in df.columns)
    print("Message Type:", type(message_data))
    print("Message:", message_data)
    producer.send(KAFKA_TOPIC, message_data)
    time.sleep(1)

print("Kafka Producer Application Completed.")

Kafka Producer Application Started ... 
Message Type: <class 'str'>
Message: +Y7DNNNCRLxa9tHiluVGkYY5B2W/n3ZpLplkJINLIdM=,KL0255921,SBA,SBPPS,1999-03-10,None,0
Message Type: <class 'str'>
Message: U7RRmweke5W8oVRNvaKXpZJPf6MTWIf+3pVCi/uDj3w=,KL0255937,SBA,SBPPS,1999-03-10,None,0
Message Type: <class 'str'>
Message: U7RRmweke5W8oVRNvaKXpW7opsj+z4fEE98TMsDU5ZQ=,KL0255938,SBA,SBPPS,1999-03-10,None,0
Message Type: <class 'str'>
Message: Khed5di6hfdejAOOBP6XBG7opsj+z4fEE98TMsDU5ZQ=,KL0255948,SBA,SBANU,1999-03-22,None,0
Message Type: <class 'str'>
Message: wt0lfnWsGTalCHEtZvy671U78SwdSDGmCdZQ2GwRpAw=,KL0255976,SBA,SBPPS,1999-04-22,None,0
Message Type: <class 'str'>
Message: wt0lfnWsGTalCHEtZvy675JPf6MTWIf+3pVCi/uDj3w=,KL0255977,SBA,SBPPS,1999-04-22,None,0
Message Type: <class 'str'>
Message: wt0lfnWsGTalCHEtZvy6727opsj+z4fEE98TMsDU5ZQ=,KL0255978,SBA,SBPPS,1999-04-22,None,0
Message Type: <class 'str'>
Message: FtDcN60NxCHnK87AgC9WyxWfZKBFMtej55gj9Qs8d94=,KL0255983,SBA,SBPPS,1999-04-22,None,0
